# ОПИСАНИЕ РЕШЕНИЯ

**КЛЮЧЕВАЯ ПРОБЛЕМА АЛГОТРЕЙДЕРА**

- наличие только одной реализации процесса колебаний цен финансового актива
- при постоянно растущего числа фичей, оверфит "стратегии" превращается в дикую головную боль

**ИЛЛЮСТРАЦИЯ ПРОБЛЕМЫ: ТОРГОВАЯ СТРАТЕГИЯ**

Подойдем к этому цинично и честно расскажем, как мы зафитили стратегию:
- использовали ограниченный набор известных показателей и [5] параметров из AlgoPak;
- загрузили все параметры в RandomForestRegressor и безжалостно зафитили in-sample до откровенно стыдного уровня;
- мы также ради любопытства зафитили LSTM нейронную сеть (число параметров перевалило за 5 миллионов, при наличии всего 900 торговых дней) с аналогичным результатом;
- некоторые инструменты при этом, условно успешно переживают out-of-sample период: accuracy > 50%.
- эту стратегия успешно запущена в нашем симуляторе, может демонстрировать сделки, и, возможно, даже останется в некотором "плюсе" [ИЛЛЮСТРАЦИЯ СТРАТЕГИИ]

**РЕШЕНИЕ КЛЮЧЕВОЙ ПРОБЛЕМЫ: СЕРВИС ДЛЯ ИНВЕСТОРОВ**

Предлагаемый инвестиционный сервис ориентирован на аудиторию алготрейдеров и состоит из трех главных элементов, каждый из которых доступен по API:

1. Консолидатор рыночных данных и данных AlgoPak в объект, который мы назвали GigaCandles: [КОРОТКОЕ РЕЗЮМЕ]
2. Стандартные технические индикаторы для удобства пользования
3. И, ключевой элемент: синтетические рыночные данные

**СИНТЕТИЧЕСКИЕ ДАННЫЕ: СУТЬ ПОДХОДА**

Мы предлагаем воспользоваться нашим сервисом генерации синтетических данных по любому финансовому инструменту.

Ключевые особенности нашего подхода:
- **Сохранение зависимостей:** все темпоральные и кросс-темпоральные характеристики временных рядов полностью сохраняются, как линейные, так и любые иные;
- **Корреляция с исходным рядом:** практически 100%, будет снижаться при роста числа блоков разбиения для бутстрэпа. Размер блока определяется пользователем и зависит от характеристик (памяти) стратегии;
- **Эргодическая гипотеза:** сгенеренные сервисом альтернативные реализации цен, например SBER, обладают уникальным свойством - их средние показатели в точности соответствуют исходному ряду. По сути, речь идет о доступе к "параллельным вселенным", где торгуется выбранный финансовый инструмент;

**СИНТЕТИЧЕСКИЕ ДАННЫЕ: КАК ПОЛЬЗОВАТЬСЯ**

Сервис рекомендуется использовать одним из двух способов:

1. По итогам разработанной стратегии, сгенерить как минимум 10 синтетических траекторий (как аппроксимация 10 децилей), на которых как на дополнительном out of sample прогнать стратегию. При существенной разнице между исходным Шарпом и средним распределения из 10 Шарпов по синтетическим траекториям - красный флаг.

2. Использовать синтетические данные изначально - несколько разных генераций при необходимости - для разработки стратегии. По итогам такого подхода использовать настоящую реализацию цены как out of sample период в полном объеме.

**СИНТЕТИЧЕСКИЕ ДАННЫЕ: ОГРАНИЧЕНИЯ**

- сервис лучше всего подходит для стратегий, построенных так или иначе из цен и их производных. Фундаментальные данные требуют отдельного сервиса и решения, куда более сложного, чем то, что предлагается;
- требуется определенная осторожность при бутстрепе многомерных векторов, состоящих не только из цен закрытия: важно проверять полученные синтетические данные на сохранения внутри-векторных соотношений;
- и главное: даже безграниченый объем синтетических данных не является гарантией от оверфита! не теряйте голову!

In [ ]:
import numpy as np
import pandas as pd
import time as tm
import datetime as dt
import tensorflow as tf

# Data preparation
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from collections import deque

# AI
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Input, Conv2D, MaxPool2D, Flatten
from tensorflow.keras import regularizers
from keras.models import Sequential

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/Data/sber.csv'
path2 = '/content/drive/MyDrive/Data/sber_price.csv'
path3 = '/content/drive/MyDrive/Data/sber_large.csv'

In [ ]:
df = pd.read_csv(path, parse_dates = ['date'], index_col = 'date')
df.index.name = None
df = df.drop(df.columns[[0, 1]], axis = 1)

In [ ]:
price = pd.read_csv(path2, parse_dates = ['date'], index_col = 'date')
price.index.name = None
price = price.drop(price.columns[[0, 1]], axis = 1)

In [ ]:
frame = pd.concat([df, price], axis = 1)
frame = frame.drop(columns = ['field'])

#LARGE DATAFRAME FOR LARGE LSTM NN

In [ ]:
df = pd.read_csv(path3, index_col = [0])

In [ ]:
df = df.copy()
df.loc[:,'ret'] = df.close.pct_change(1)
df.loc[:,'next_ret'] = df.ret.shift(-1)
df = df.dropna()
df.loc[:,'label'] = np.sign(df.next_ret)

In [ ]:
total_fields = df.columns.to_list()
fields_need = ['RSI_2', 'EMA_RSI_14', 'RSI_50', 'adx_5', 'adx_10', 'macd_signal', 'macd_osc']

In [ ]:
fields_out = ['next_ret', 'label']
total_fields = fields_need
feats_cols = [x for x in total_fields if x not in fields_out]

In [ ]:
dim_f = len(feats_cols)

In [ ]:
feats = np.column_stack([x for y, x in df[feats_cols].items()])
labels = np.column_stack([x for y, x in df['label'].items()]).ravel()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(feats, labels, test_size = 200)

In [ ]:
# For n-size time steps in the model
#steps = 5
#dim1 = int(x_train.shape[0] / steps)
#dim2 = int(x_train.shape[1])
#x_train_prcs = x_train.reshape(dim1, steps, dim2)

In [ ]:
# For single time step in the model
x_train_prcs = x_train.reshape(-1,1,dim_f)

In [ ]:
def GetTrainedModel(x_train, y_train, dim = dim_f):

  y_train = (y_train + 1)/2

  model = Sequential()
  model.add(LSTM(512, activation = 'tanh', return_sequences=True, input_shape = (1, dim)))
  model.add(Dropout(0.5))
  model.add(LSTM(128, activation = 'tanh'))
  model.add(Dropout(0.5))
  model.add(Dense(32, activation = 'tanh'))
  model.add(Dense(1, activation = 'sigmoid'))

  BATCH_SIZE = 64
  EPOCHS = 2250

  model.compile(loss='binary_crossentropy', optimizer='adam')

  model.fit(x_train, y_train,
            batch_size=BATCH_SIZE,
            epochs=EPOCHS,
            verbose=1)

  model.summary()

  return model

In [ ]:
model = GetTrainedModel(x_train_prcs, y_train)

Epoch 1/2250
12/12 [==============================] - 6s 34ms/step - loss: 0.6994
Epoch 2/2250
12/12 [==============================] - 0s 35ms/step - loss: 0.6923
Epoch 3/2250
12/12 [==============================] - 0s 34ms/step - loss: 0.6897
Epoch 4/2250
12/12 [==============================] - 0s 34ms/step - loss: 0.6894
Epoch 5/2250
12/12 [==============================] - 0s 33ms/step - loss: 0.6922
Epoch 6/2250
12/12 [==============================] - 0s 35ms/step - loss: 0.6967
Epoch 7/2250
12/12 [==============================] - 0s 34ms/step - loss: 0.6908
Epoch 8/2250
12/12 [==============================] - 0s 33ms/step - loss: 0.6870
Epoch 9/2250
12/12 [==============================] - 0s 34ms/step - loss: 0.6855
Epoch 10/2250
12/12 [==============================] - 0s 33ms/step - loss: 0.6865
Epoch 11/2250
12/12 [==============================] - 0s 32ms/step - loss: 0.6847
Epoch 12/2250
12/12 [==============================] - 0s 34ms/step - loss: 0.6872
Epoch 13/2250

In [ ]:
model.save('/content/drive/MyDrive/Data/MOEX_LSTM_Model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
modelAccuracy(model, x_train, x_test, y_train, y_test)

7/7 [==============================] - 0s 10ms/step
Shapes and types:
y_train_binary: (726,) float64
train_predictions: (726,) int64
y_test_binary: (200,) float64
test_predictions: (200,) int64
Training Accuracy: 0.9256198347107438
Test Accuracy: 0.53

Classification Report for Training Data:
              precision    recall  f1-score   support

         0.0       0.93      0.91      0.92       345
         1.0       0.92      0.94      0.93       381

    accuracy                           0.93       726
   macro avg       0.93      0.92      0.93       726
weighted avg       0.93      0.93      0.93       726


Classification Report for Test Data:
              precision    recall  f1-score   support

         0.0       0.53      0.48      0.51       100
         1.0       0.53      0.58      0.55       100

    accuracy                           0.53       200
   macro avg       0.53      0.53      0.53       200
weighted avg       0.53      0.53      0.53       200


Confusion Mat

In [ ]:
def modelAccuracy(model, x_train, x_test, y_train, y_test, dim = dim_f):

  train_predictions = model.predict(x_train.reshape(-1,1,dim)).flatten()
  test_predictions = model.predict(x_test.reshape(-1,1,dim)).flatten()

  # Convert predictions to label format (0 or 1)
  train_predictions = np.where(train_predictions < 0.5, 0, 1)
  test_predictions = np.where(test_predictions < 0.5, 0, 1)

  # Convert original y_train and y_test from -1 and 1 to 0 and 1
  y_train_binary = (y_train + 1.0001) // 2
  y_test_binary = (y_test + 1.0001) // 2

  print("Shapes and types:")
  print("y_train_binary:", y_train_binary.shape, y_train_binary.dtype)
  print("train_predictions:", train_predictions.shape, train_predictions.dtype)
  print("y_test_binary:", y_test_binary.shape, y_test_binary.dtype)
  print("test_predictions:", test_predictions.shape, test_predictions.dtype)

  # Calculate accuracy
  train_accuracy = accuracy_score(y_train_binary, train_predictions)
  test_accuracy = accuracy_score(y_test_binary, test_predictions)

  # Print accuracy
  print("Training Accuracy:", train_accuracy)
  print("Test Accuracy:", test_accuracy)

  # Additional metrics
  print("\nClassification Report for Training Data:")
  print(classification_report(y_train_binary, train_predictions))

  print("\nClassification Report for Test Data:")
  print(classification_report(y_test_binary, test_predictions))

  # Confusion Matrix
  print("\nConfusion Matrix for Training Data:")
  print(confusion_matrix(y_train_binary, train_predictions))

  print("\nConfusion Matrix for Test Data:")
  print(confusion_matrix(y_test_binary, test_predictions))

In [ ]:
def GetCNN_Model(seq_len=1, n_features=dim_f, n_filters=(2, 8, 8), droprate=0.2):
    """2D-CNNpred model according to the paper."""
    model = Sequential()
    model.add(Input(shape=(1, n_features, 1)))
    model.add(Conv2D(n_filters[0], kernel_size=(1, n_features), activation="relu"))
    model.add(Conv2D(n_filters[1], kernel_size=(1, 1), activation="tanh"))
    # Removed one Conv2D and MaxPool2D layers
    model.add(Flatten())
    model.add(Dropout(droprate))
    model.add(Dense(1, activation="sigmoid"))

    return model

In [ ]:
x_train_prcs_cnn = x_train.reshape(-1, 1, dim_f, 1)
y_train_cnn = (y_train + 1.0001)//2
y_test_cnn = (y_test + 1.0001) //2

In [ ]:
model = GetCNN_Model()
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["acc"])
model.fit(x_train_prcs_cnn, y_train_cnn, epochs = 500, batch_size = 64)

Epoch 1/500
12/12 [==============================] - 3s 8ms/step - loss: 0.7493 - acc: 0.4807
Epoch 2/500
12/12 [==============================] - 0s 9ms/step - loss: 0.7354 - acc: 0.5000
Epoch 3/500
12/12 [==============================] - 0s 7ms/step - loss: 0.7251 - acc: 0.5207
Epoch 4/500
12/12 [==============================] - 0s 8ms/step - loss: 0.7397 - acc: 0.5041
Epoch 5/500
12/12 [==============================] - 0s 7ms/step - loss: 0.7300 - acc: 0.5014
Epoch 6/500
12/12 [==============================] - 0s 7ms/step - loss: 0.7356 - acc: 0.5083
Epoch 7/500
12/12 [==============================] - 0s 8ms/step - loss: 0.7450 - acc: 0.4862
Epoch 8/500
12/12 [==============================] - 0s 10ms/step - loss: 0.7170 - acc: 0.5165
Epoch 9/500
12/12 [==============================] - 0s 8ms/step - loss: 0.7207 - acc: 0.5289
Epoch 10/500
12/12 [==============================] - 0s 6ms/step - loss: 0.7252 - acc: 0.5248
Epoch 11/500
12/12 [==============================] - 0s 7

In [ ]:
train_predictions_cnn = model.predict(x_train.reshape(-1,1,dim_f, 1)).flatten()
test_predictions_cnn = model.predict(x_test.reshape(-1,1,dim_f, 1)).flatten()

7/7 [==============================] - 0s 2ms/step


In [ ]:
train_accuracy = accuracy_score(y_train_cnn, np.where(train_predictions_cnn > 0.5, 1, 0))
test_accuracy = accuracy_score(y_test_cnn, np.where(test_predictions_cnn > 0.5, 1, 0))

In [ ]:
train_accuracy, test_accuracy

(0.5743801652892562, 0.495)